# Descripción del proyecto

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del sprint de Análisis estadístico de datos). Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.

Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.


# Introducción

En el area de las telecomunicaciones es importante la adaptación a las necesidades de los clientes para mantener una ventaja competitiva. La compañía móvil Megaline ha identificado una oportunidad para optimizar la satisfacción del cliente y mejorar la eficiencia de sus ofertas de planes mediante el uso de modelos predictivos. Actualmente, muchos de sus clientes continúan utilizando planes antiguos, lo que podría limitar el valor que reciben tanto ellos como la empresa.

la compañía desea desarrollar un modelo predictivo que analice el comportamiento de los clientes y recomiende el plan más adecuado. Utilizando datos históricos de clientes que ya han cambiado a los nuevos planes, el objetivo es crear un modelo de clasificación con una exactitud mínima de 0.75. Este modelo permitirá personalizar las recomendaciones y mejorar la satisfacción del cliente, facilitando una transición hacia las nuevas ofertas.



# Inicialización

In [3]:
# Cargamos todas las librerias que creemos que vamos a utilizar

import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

## Cargar datos

In [4]:
# cargamos el archivo en un dataframe

df = pd.read_csv('/datasets/users_behavior.csv')

In [18]:
#validamos las dimensiones del dataframe (usamos shape) e imprimimos las primeras filas (usamos head)

print(df.shape)
print(df.head())
print()

#mostramos la informacion del dataframe con el metodo info
df.info()

(3214, 5)
   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


### Duplicados

In [15]:
# se valida si tenemos duplicados en el dataframe con el metodo duplicated.
print(df.duplicated().sum())

0


### Valores ausentes

In [16]:
# se valida si tenemos valores ausentes en el dataframe con el metodo isna.
print(df.isna().sum())

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64


### Comportamiento estadístico de las variables.

In [20]:
# Mostramos un resumen estadístico de las variables utilizando el metodo describe
print(df.describe())

             calls      minutes     messages       mb_used     is_ultra
count  3214.000000  3214.000000  3214.000000   3214.000000  3214.000000
mean     63.038892   438.208787    38.281269  17207.673836     0.306472
std      33.236368   234.569872    36.148326   7570.968246     0.461100
min       0.000000     0.000000     0.000000      0.000000     0.000000
25%      40.000000   274.575000     9.000000  12491.902500     0.000000
50%      62.000000   430.600000    30.000000  16943.235000     0.000000
75%      82.000000   571.927500    57.000000  21424.700000     1.000000
max     244.000000  1632.060000   224.000000  49745.730000     1.000000


Se valida el dataframe mostrando la informacion con el metodo info, en el cual evidenciamos que los tipos de datos de cada columna son correctos. adicionalmente se revisa si tenemos duplicados o valores ausentes lo cual nos da cero para ambos aspectos, ademas se realiza un resumen estadistico del dataframe llamando al metodo describe, en el cual podemos ver informacion estadistica como el promedio, valores minimos y maximos, desviacion estandar, etc.

## Segmentación de datos

In [7]:
# establecemos variables separadas para nuestras caracteristicas y para nuestro objetivo

features = df.drop(['is_ultra'], axis= 1)
target = df['is_ultra']

In [8]:
# no tenemos conjunto de prueba. los datos fuentes se dividiran en 3(entrenamiento, validacion y prueba).

# dividimos el entrenamiento (60%) y un conjunto combinado de validación/prueba (40%)
features_train, features_temp, target_train, target_temp = train_test_split(features, target, test_size=0.40, random_state=12345)

# ahora dividimos el conjunto combinado (40%) en la mitad para obtener validacion 20% y prueba 20%
features_val, features_test, target_val, target_test = train_test_split(features_temp, target_temp, test_size=0.50, random_state=12345)


## Calidad de diferentes modelos

### Modelo árbol de decisión

In [9]:
#iteraremos diferentes valores del hiperparametro max_depth con un bucle y compararemos la calidad de las diferentes versiones

for depth in range(1, 6):
        model_1 = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
        model_1.fit(features_train, target_train)  # entrenamos el modelo

        predictions_val = model_1.predict(features_val) # encontramos las predicciones usando el conjunto de validación

        print("max_depth =", depth, ": ", end='')
        print(accuracy_score(target_val, predictions_val))

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944


### Modelo bosque aleatorio

In [10]:
# hacemos un bucle que pruebe modelos de bosque aleatorio con varios números de estimadores.

best_score = 0
best_est = 0
for est in range(1, 50): # seleccionamos el rango del hiperparámetro
    model_2 = RandomForestClassifier(random_state=54321, n_estimators= est) # configuramos el número de árboles
    model_2.fit(features_train, target_train) # entrenamos el modelo
    
    score = model_2.score(features_val, target_val) # calculamos la puntuación de accuracy en el conjunto de validación
    if score > best_score:
        best_score = score # guarda la mejor puntuación de accuracy en el conjunto de validación
        best_est = est # guarda el número de estimadores que corresponden a la mejor puntuación de exactitud

print("La exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))

La exactitud del mejor modelo en el conjunto de validación (n_estimators = 40): 0.80248833592535


### Modelo regresión logística

In [11]:
# Iniciamos el modelo de regresion logistica con los parámetros random_state=54321 y solver='liblinear'
model_3 = LogisticRegression(random_state=54321, solver='liblinear') 

model_3.fit(features_train, target_train) # entrenamos el modelo
score_train = model_3.score(features_train, target_train) # calculamos la puntuación de accuracy en el conjunto de entrenamiento
score_val = model_3.score(features_val, target_val) # calculamos la puntuación de accuracy en el conjunto de validación

print("Accuracy del modelo de regresión logística en el conjunto de entrenamiento:", score_train)
print("Accuracy del modelo de regresión logística en el conjunto de validación:", score_val)

Accuracy del modelo de regresión logística en el conjunto de entrenamiento: 0.7505186721991701
Accuracy del modelo de regresión logística en el conjunto de validación: 0.7589424572317263


El modelo de bosque aleatorio con n_estimators=40 resultó ser el modelo más preciso en este experimento, logrando una exactitud de 0.8024 en el conjunto de validación. Este indicaria que este modelo tiene una mejor capacidad para capturar las relaciones complejas en los datos, debido a la combinación de múltiples árboles de decisión.

El árbol de decisión fue también competitivo, con un buen rendimiento en max_depth=3, pero no alcanzó la precisión del bosque aleatorio.

La regresión logística fue el modelo menos preciso en este caso, aunque rápida y eficiente, no se ajustó tan bien como los otros modelos más complejos, posiblemente debido a la naturaleza no lineal de los datos.

De acuero a lo anterior, se recomendaría utilizar el modelo de bosque aleatorio con n_estimators=40 para la evaluación final en el conjunto de prueba.


## Calidad del modelo usando el conjunto de prueba

In [12]:
# Entrenamos el modelo de bosque aleatorio con n_estimators=40

final_model = RandomForestClassifier(random_state=54321, n_estimators=40)
final_model.fit(features_train, target_train)  # Entrenamos el modelo con el conjunto de entrenamiento
   
# Hacemos predicciones con el conjunto de prueba
predictions_test = final_model.predict(features_test)

# Calculamos la precisión del modelo en el conjunto de prueba
test_accuracy = accuracy_score(target_test, predictions_test)

# Mostramos la precisión
print(f"Exactitud del modelo de bosque aleatorio en el conjunto de prueba: {test_accuracy:}")

Exactitud del modelo de bosque aleatorio en el conjunto de prueba: 0.7931570762052877


## Prueba de cordura al modelo

Para asegurar que nuestro modelo entrenado tiene valor real y no está simplemente haciendo predicciones aleatorias podemos hacer la validacion con una prueba de cordura. Podriamos utilizar DummyClassifier que siempre predice la clase más frecuente (valor mas frecuente de nuestra variable objetivo), y comparamos su rendimiento con nuestro modelo real.

Si nuestro modelo de bosque aleatorio supera claramente al modelo dummy, podemos decir que nuestro modelo de bosque aleatorio no está haciendo predicciones aleatorias e indicaria que si esta capturando patrones en los datos


In [13]:
# Creamos un modelo "Dummy" que siempre prediga la clase mas frecuente de nuestra variable objetivo (is_ultra)
from sklearn.dummy import DummyClassifier

dummy_model = DummyClassifier(strategy='most_frequent', random_state=12345)
dummy_model.fit(features_train, target_train) # Entrenamos el modelo con el conjunto de entrenamiento

dummy_score = dummy_model.score(features_test, target_test) # evaluamos en el conjunto de prueba

print(f'Exactitud del modelo Dummy: {dummy_score}')

Exactitud del modelo Dummy: 0.6842923794712286


# Conclusiones

El modelo de bosque aleatorio fue el mejor para la tarea de clasificación entre los planes Smart y Ultra, con una exactitud consistente tanto en los conjuntos de validación como de prueba.

La exactitud final del 0.793 en el conjunto de prueba es adecuada para las necesidades de la compañía, ya que superó el umbral de 0.75 exigido.

al comparar con el clasificador dummy demuestra que el modelo realmente ha aprendido a diferenciar entre las dos clases basándose en las características del comportamiento de los usuarios.

el modelo de bosque aleatorio es una herramienta efectiva para predecir si un usuario debería tener el plan Smart o Ultra, ya que se basa en sus patrones de uso de llamadas, mensajes y datos. Esto ayudará a Megaline para orientar a los usuarios hacia el plan más adecuado para ellos.
